# Libraries

In [1]:
import pandas as pd
import re

# Load Data

In [2]:
# Load games requirements table
games_requirement_data = pd.read_csv('../row_tables/system_requirement_website_data.csv', index_col=0)

# Load benchmark score table
benchmark_score_data = pd.read_csv('../row_tables/benchmark_score_data.csv', index_col=0)

# Load GPU table
gpu_data = pd.read_csv('../row_tables/gpu_data.csv', index_col=0)

# Load RAM table
ram_data = pd.read_csv('../row_tables/ram_data.csv', index_col=0)

# Standardize GPU names functions

In [3]:
def standardize_nvidia_gpus(df, column):
    # Regex pattern to identify NVIDIA GPU models and components
    pattern = re.compile(r"""
        (?:NVIDIA\s*)?          # Optional "NVIDIA" prefix
        (?:GeForce\s+)?         # Optional "GeForce" followed by whitespace
        (GTX|RTX|GT|Quadro|TITAN)\s*  # Captured model family (GTX, RTX, etc.)
        (?:[-®™]?\s*)?         # Optional special characters
        (\d{2,4})              # Numerical part (e.g., 9800, 1660, 1080 Ti)
        \s*                     # Optional whitespace after number
        (SUPER|Ti|Laptop\sGPU|Max-Q)?  # Optional suffixes (case-insensitive)
    """, re.IGNORECASE | re.VERBOSE)
    
    def extract_and_standardize(gpu_str):
        if not isinstance(gpu_str, str):
            return None
        
        gpu_str = re.sub(r"VIDEO CARD:\s*", "", gpu_str, flags=re.IGNORECASE)
        matches = pattern.findall(gpu_str)
        
        standardized_cards = []
        for series, number, suffix in matches:
            # Determine product line based on model family
            series_upper = series.upper()
            if series_upper in ['GTX', 'RTX', 'GT']:
                product_line = 'GeForce'
            elif series_upper == 'QUADRO':
                product_line = 'Quadro'
            elif series_upper == 'TITAN':
                product_line = 'TITAN'
            else:
                product_line = 'GeForce'  # Default to GeForce
            
            base_name = f"NVIDIA {product_line} {series_upper} {number}"
            
            # Process suffix
            if suffix:
                suffix_lower = suffix.strip().lower()
                if suffix_lower == 'ti':
                    standardized_suffix = 'Ti'
                elif suffix_lower == 'super':
                    standardized_suffix = 'SUPER'
                elif suffix_lower == 'laptop gpu':
                    standardized_suffix = 'Laptop GPU'
                elif suffix_lower == 'max-q':
                    standardized_suffix = 'Max-Q'
                else:
                    standardized_suffix = suffix.strip().title()
                base_name += f" {standardized_suffix}"
            
            standardized_cards.append(base_name)
        
        return standardized_cards[0] if standardized_cards else None
    
    df[column] = df[column].apply(extract_and_standardize)
    return df

# Standardize RAM amount function

In [4]:
# Function to extract only numbers and append "GB"
def standardize_ram(value):
    num = ''.join(filter(str.isdigit, value))  # Keep only digits
    return f"{num} GB" if num else value  # Append "GB" if numbers found

# Aplly all changes

In [5]:
# Apply changes for GPU
games_requirement_data = standardize_nvidia_gpus(games_requirement_data, 'gpu_minimum')
games_requirement_data = standardize_nvidia_gpus(games_requirement_data, 'gpu_recommended')
benchmark_score_data = standardize_nvidia_gpus(benchmark_score_data, 'gpu_name')

# Apply changes for RAM
games_requirement_data['ram_minimum'] = games_requirement_data['ram_minimum'].apply(standardize_ram)
games_requirement_data['ram_recommended'] = games_requirement_data['ram_recommended'].apply(standardize_ram)

# Handling Null Values

In [6]:
def null_values(df):
    for column in df.columns:
        null_count = df[column].isna().sum()
        if (null_count>0):
            print(f'{column}: {null_count} null values')    

In [7]:
null_values(games_requirement_data)

gpu_minimum: 10 null values
gpu_recommended: 4 null values


In [8]:
null_values(benchmark_score_data)

gpu_name: 18 null values


In [9]:
games_requirement_data.dropna(inplace=True)
benchmark_score_data.dropna(inplace=True)

# Handling Duplicates

In [10]:
benchmark_score_data.drop_duplicates(subset='gpu_name', keep='first', inplace=True)

# Handling GPU Market Share
Since only NVIDIA was filtered, the market share is currently less than 100%. However, it should be adjusted to 100% to represent the total share of NVIDIA GPUs.

In [11]:
# Convert market share to numeric
gpu_data['market_share'] = gpu_data['market_share'].str.rstrip('%').astype(float)/100

In [12]:
# Sum market share
print(f"Before:\n{round(gpu_data['market_share'].sum(), 3)}")

Before:
0.793


In [13]:
# Normalizing market share
gpu_data['market_share'] = gpu_data['market_share']/(gpu_data['market_share'].sum())

In [14]:
# Sum market share
print(f"After:\n{round(gpu_data['market_share'].sum(), 3)}")

After:
1.0


# Handling RAM Market Share
Just like the GPU, the market share is currently less than 100%. Since "Less than 4GB" and "More than 64GB" was removed from ram_data, so it's necessary adjust to 100%

In [15]:
# Convert market share to numeric
ram_data['market_share'] = ram_data['market_share'].str.rstrip('%').astype(float)/100

In [16]:
# Sum market share
print(f"Before:\n{round(ram_data['market_share'].sum(), 3)}")

Before:
0.986


In [17]:
# Normalizing market share
ram_data['market_share'] = ram_data['market_share']/(ram_data['market_share'].sum())

In [18]:
# Sum market share
print(f"After:\n{round(ram_data['market_share'].sum(), 3)}")

After:
1.0


# Join Benchmark Score on GPU table
Fill the GPU table with all data avaiable

In [19]:
# Minimum GPU required
gpu_data = pd.merge(gpu_data, benchmark_score_data, how='left', on='gpu_name')

# Join Benchmark Score on Requirements table
The goal is to measure only the GPU power, so it's not necessary to keep the names of the minimum and recommended requirements

In [20]:
# Minimum GPU required
games_requirement_data = pd.merge(games_requirement_data, benchmark_score_data, how='left', left_on='gpu_minimum', right_on='gpu_name')
games_requirement_data.rename(columns={'benchmark_score':'gpu_minimum_benchmark_score'}, inplace=True)

# Drop column
games_requirement_data.drop('gpu_name', axis=1, inplace=True)

In [21]:
# Recommended GPU required
games_requirement_data = pd.merge(games_requirement_data, benchmark_score_data, how='left', left_on='gpu_recommended', right_on='gpu_name')
games_requirement_data.rename(columns={'benchmark_score':'gpu_recommended_benchmark_score'}, inplace=True)

# Drop column
games_requirement_data.drop('gpu_name', axis=1, inplace=True)

In [22]:
# Drop GPU Column
games_requirement_data.drop(['gpu_minimum', 'gpu_recommended'], axis=1, inplace=True)

# Transform RAM amount to numeric

In [23]:
# Extract RAM amount and converts to numeric
ram_data['ram'] = ram_data['ram'].str.extract(r'(\d+)').astype(int)
games_requirement_data['ram_minimum'] = games_requirement_data['ram_minimum'].str.extract(r'(\d+)').astype(int)
games_requirement_data['ram_recommended'] = games_requirement_data['ram_recommended'].str.extract(r'(\d+)').astype(int)

# Save Data

In [24]:
# Save games requirements table
games_requirement_data.to_csv('../tables/games_requirement_data.csv', index=True)

# Save benchmark score table
benchmark_score_data.to_csv('../tables/benchmark_score_data.csv', index=True)

# Save gpu data table
gpu_data.to_csv('../tables/gpu_data.csv', index=True)

# Save ram data table
ram_data.to_csv('../tables/ram_data.csv', index=True)